In [55]:
import math
import numpy as np
from scipy.signal import find_peaks

In [56]:
DATA_SOURCE = "June 27 smaller dist 3 plate"

header_lines = 8
data_lines = 1024

prominence = 30
width = 10

In [57]:
def get_data_sets(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        total_lines = len(lines)
        total_data_sets = math.floor(total_lines / (header_lines + data_lines))
        file.close()

    return total_data_sets - 1

In [58]:
def read_data_from_file(file_path):
    animation = "|/-\\"
    tme = 0

    with open(file_path, "r") as file:
        # Skip the initial 8 header lines
        for _ in range(header_lines):
            next(file)

        # Initialize an empty list to store the data
        data = []

        # Read the data sets
        for _ in range(ds):
            # Read the data lines
            data_set = [next(file).strip().split() for _ in range(data_lines)]
            data_set = [float(item) for sublist in data_set for item in sublist]
            data.append(data_set)

            print(animation[tme % len(animation)], end="\r")
            tme += 1

            # Skip the next 8 header lines
            for _ in range(header_lines):
                next(file)

    # Convert the list of data into a NumPy array
    data_arr = np.array(data)
    return data_arr

In [59]:
# Scintillator PMTs
trigger_file_path = "DATA/" + DATA_SOURCE + "/TR_0_0.txt"
distant_plate_file_path = "DATA/" + DATA_SOURCE + "/wave_0.txt"
bottom_plate_file_path = "DATA/" + DATA_SOURCE + "/wave_2.txt"

print("start")
ds = get_data_sets(trigger_file_path)

trigger_data_array = read_data_from_file(trigger_file_path)
print("1")

distant_plate_data_array = read_data_from_file(distant_plate_file_path)
print("2")

bottom_plate_data_array = read_data_from_file(bottom_plate_file_path)
print("3")

data_arr_scints = np.array(
    [trigger_data_array, bottom_plate_data_array, distant_plate_data_array]
)

# data_arr_cher = np.array([data_array_3, data_array_4, data_array_5, data_array_6, data_array_7])
print("done")

start
1
2
3
done


In [60]:
trigger_peaks = []
bottom_peaks = []
dist_peaks = []

for i, x1 in enumerate(trigger_data_array):

    peak1, properties1 = find_peaks(-x1, prominence=prominence, width=width)
    peak2, properties2 = find_peaks(-bottom_plate_data_array[i], prominence=prominence, width=width)
    peak3, properties3 = find_peaks(-distant_plate_data_array[i], prominence=prominence, width=width)

    if peak1.size == 0:
        trigger_peaks.append(2000)
    else:
        trigger_peaks.append(peak1[0])

    if peak2.size == 0:
        bottom_peaks.append(4000)
    else:
        bottom_peaks.append(peak2[0])

    if peak3.size == 0:
        dist_peaks.append(6000)
    else:
        dist_peaks.append(peak3[0])

In [61]:
box_index = np.array([
    i for i, x in enumerate(np.isclose(trigger_peaks, bottom_peaks, atol=1024)) if x
])
dist_index = np.array([
    i for i, x in enumerate(np.isclose(trigger_peaks, dist_peaks, atol=1024)) if x
])

both_index = box_index[np.isin(box_index, dist_index)]

print(ds)
print(len(box_index))
print(len(dist_index))
print(len(both_index))

11004
3522
544
246


In [62]:
with open("DATA/" + DATA_SOURCE + "/both/trigger.txt", "w") as txt_file:
    for idx in both_index:
        for data in data_arr_scints[0][idx]:
            txt_file.write(
                "".join(str(data)) + "\n"
            )  # works with any number of elements in a line
with open("DATA/" + DATA_SOURCE + "/both/bottom.txt", "w") as txt_file:
    for idx in both_index:
        for data in data_arr_scints[1][idx]:
            txt_file.write(
                "".join(str(data)) + "\n"
            )  # works with any number of elements in a line
with open("DATA/" + DATA_SOURCE + "/both/distant.txt", "w") as txt_file:
    for idx in both_index:
        for data in data_arr_scints[2][idx]:
            txt_file.write(
                "".join(str(data)) + "\n"
            )  # works with any number of elements in a line

In [63]:
def read_selected_data_from_file(file_path, index):
    animation = "|/-\\"
    tme = 0

    skip_sets = np.ediff1d(index)

    with open(file_path, "r") as file:
        # Initialize an empty list to store the data
        data = []

        for _ in range(((header_lines + data_lines) * (index[0] + 1) + header_lines)):
            next(file)

        # Read the data lines
        data_set = [next(file).strip().split() for _sets in range(data_lines)]
        data_set = [float(item) for sublist in data_set for item in sublist]
        data.append(data_set)
        tme += 1

        # Read the data sets
        for _sets in skip_sets:
            # Skip to the next interesting dataset
            for _ in range(((header_lines + data_lines) * (_sets - 1) + header_lines)):
                next(file)

            # Read the data lines
            data_set = [next(file).strip().split() for _sets in range(data_lines)]
            data_set = [float(item) for sublist in data_set for item in sublist]
            data.append(data_set)

            print(animation[tme % len(animation)], end="\r")
            tme += 1

    # Convert the list of data into a NumPy array
    data_arr = np.array(data)
    print("Done")
    return data_arr

In [64]:
# Cherenkov PMTs
file_path_3 = "DATA/" + DATA_SOURCE + "/wave_3.txt"
file_path_4 = "DATA/" + DATA_SOURCE + "/wave_4.txt"
file_path_5 = "DATA/" + DATA_SOURCE + "/wave_5.txt"
file_path_6 = "DATA/" + DATA_SOURCE + "/wave_6.txt"
file_path_7 = "DATA/" + DATA_SOURCE + "/wave_7.txt"

data_array_3 = read_selected_data_from_file(file_path_3, both_index)
data_array_4 = read_selected_data_from_file(file_path_4, both_index)
data_array_5 = read_selected_data_from_file(file_path_5, both_index)
data_array_6 = read_selected_data_from_file(file_path_6, both_index)
data_array_7 = read_selected_data_from_file(file_path_7, both_index)

Done
Done
Done
Done
Done


In [65]:
with open("DATA/" + DATA_SOURCE + "/both/short_3.txt", "w") as txt_file:
    for sets in data_array_3:
        for data in sets:
            txt_file.write(
                "".join(str(data)) + "\n"
            )  # works with any number of elements in a line

with open("DATA/" + DATA_SOURCE + "/both/short_4.txt", "w") as txt_file:
    for sets in data_array_4:
        for data in sets:
            txt_file.write(
                "".join(str(data)) + "\n"
            )  # works with any number of elements in a line

with open("DATA/" + DATA_SOURCE + "/both/short_5.txt", "w") as txt_file:
    for sets in data_array_5:
        for data in sets:
            txt_file.write(
                "".join(str(data)) + "\n"
            )  # works with any number of elements in a line

with open("DATA/" + DATA_SOURCE + "/both/short_6.txt", "w") as txt_file:
    for sets in data_array_6:
        for data in sets:
            txt_file.write(
                "".join(str(data)) + "\n"
            )  # works with any number of elements in a line

with open("DATA/" + DATA_SOURCE + "/both/short_7.txt", "w") as txt_file:
    for sets in data_array_7:
        for data in sets:
            txt_file.write(
                "".join(str(data)) + "\n"
            )  # works with any number of elements in a line